In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')


In [9]:
data.drop("Unnamed: 0", axis=1).head()[:2][:2]

,links,characters,name,position 1,position 2,experince 1,experince 2,field of studies 1,field of studies 2,degree 1,degree 2,industry,skills,influencer,country,summary
0,https://nl.linkedin.com/in/mieke-zonneveld-58a...,Encourager (Is),Mieke Zonneveld,Docent Nederlands,Auteur,9.0,9.0,Opleiding leraar Nederlands in de eerste graad,Nederlandse taal- en letterkunde,Master,Master of Arts - MA,Primary/Secondary Education,[],False,Netherlands,NaN
1,https://nl.linkedin.com/in/jiska-de-ligt-2a7527b,Counselor (Si),Jiska de Ligt,HR Manager,bestuurslid (secretaris),1.0,1.0,Comparative Literature,English Language and Literature/Letters,Research master,NaN,Information Technology & Services,"['Academic Writing', 'English Literature', 'Li...",False,Netherlands,Bij Infi ben ik verantwoordelijk voor het verz...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')

# Convert non-string values to strings in the feature columns
text_features = data[['position 1', 'position 2', 'field of studies 1', 'field of studies 2', 'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(f"max_sequence------>>>>{max_sequence_length}")
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100)(embedding_layer)
output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_text, y_train, validation_data=(X_test_text, y_test),
          epochs=10, batch_size=32)

# Save the model
model.save('model.h5')


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')
cols=['position 1', 'position 2', 'field of studies 1','experince 1', 'experince 2', 'field of studies 2',
       'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']

for col in cols:
  data[col] = data[col].fillna("Unknown")
# Convert non-string values to strings in the feature columns
text_features = data[['position 1', 'position 2', "experince 1","experince 2", 
                      'field of studies 1', 'field of studies 2', 
                      'degree 1', 'degree 2', 'industry', 'skills',
                      'influencer', 'country', 'summary']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.1, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100)(embedding_layer)
output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_text, y_train, validation_data=(X_test_text, y_test),
          epochs=50, batch_size=32)

# Save the model
model.save('model2_test_size=0.1.h5')


523
Epoch 1/50
115/115 [==============================] - 65s 527ms/step - loss: 2.5251 - accuracy: 0.1548 - val_loss: 2.4070 - val_accuracy: 0.1966
Epoch 2/50
115/115 [==============================] - 59s 513ms/step - loss: 2.2792 - accuracy: 0.2105 - val_loss: 2.3431 - val_accuracy: 0.2015
Epoch 3/50
115/115 [==============================] - 59s 513ms/step - loss: 2.0298 - accuracy: 0.2768 - val_loss: 2.4085 - val_accuracy: 0.2064
Epoch 4/50
115/115 [==============================] - 59s 512ms/step - loss: 1.7504 - accuracy: 0.3808 - val_loss: 2.4702 - val_accuracy: 0.1941
Epoch 5/50
115/115 [==============================] - 59s 509ms/step - loss: 1.3977 - accuracy: 0.5242 - val_loss: 2.5675 - val_accuracy: 0.2088
Epoch 6/50
115/115 [==============================] - 59s 511ms/step - loss: 1.0272 - accuracy: 0.6631 - val_loss: 2.7992 - val_accuracy: 0.2260
Epoch 7/50
115/115 [==============================] - 58s 507ms/step - loss: 0.7348 - accuracy: 0.7767 - val_loss: 3.1143 - va

In [5]:
import datetime
import requests
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

positions0=[]
positions1=[]
experinces0=[]
experinces1=[]
skills=[]
influencer=[]
country=[]
field_of_studies0=[]
field_of_studies1=[]
degrees0=[]
degrees1=[]
industries=[]
names=[]
summaries=[]
certifications=[]

link="https://www.linkedin.com/in/ismatulla-kuyliev-43286a199/"
id = link.split('/')[4]
url = "https://api.iscraper.io/v2/profile-details"

payload = {
  'profile_id': id,
}

headers = {
  'X-API-KEY': 'hVSqiv11cY1W5YUawXUDLBn0jb4G5W44',
}

response = requests.post(url, json=payload, headers=headers)
res=response.json()
try:
  skills.append(res['skills'])
except:
  skills.append("Unknown")
position0=""
try:
  position0=res['position_groups'][0]['profile_positions'][0]['title']
  positions0.append(position0)
except:
  positions0.append("Unknown")

position1=""
try:
  position1=res['position_groups'][1]['profile_positions'][0]['title']
  positions1.append(position1)
except:
  positions1.append("Unknown")

experince0=0
try:
  experince0=res['position_groups'][0]['date']['start']['year']-res['position_groups'][0]['date']['end']['year']
  experinces0.append(experince0)
except:
  try:
    experince0=datetime.date.today().year - res['position_groups'][0]['date']['start']['year']
    experinces0.append(experince0)
  except:
    experinces0.append("Unknown")

experince1=0
try:
  experince1=res['position_groups'][1]['date']['start']['year']-res['position_groups'][1]['date']['end']['year']
  experinces1.append(experince1.abs())
except:
  try:
    experince1=datetime.date.today().year - res['position_groups'][1]['date']['start']['year']
    experinces1.append(experince1.abs())
  except:
    experinces1.append("Unknown")
      
try:
  influencer.append(res['influencer'])
except:
  influencer.append("Unknown")
try:
  country.append(res['location']['country'])
except:
  country.append("Unknown")
field_of_study0=""
try:
  field_of_study0=res['education'][0]['field_of_study']
  field_of_studies0.append(field_of_study0)
except:
  field_of_study0="Unknown"
  field_of_studies0.append(field_of_study0)

field_of_study1=""
try:
  field_of_study1=res['education'][1]['field_of_study']
  field_of_studies1.append(field_of_study1)
except:
  field_of_study1="Unknown"
  field_of_studies1.append(field_of_study1)

degree_name0=''
try:
  degree_name0=res['education'][0]['degree_name']
  degrees0.append(degree_name0)
except:
  degree_name0="Unknown"
  degrees0.append(degree_name0)

degree_name1=''
try:
  degree_name1=res['education'][1]['degree_name']
  degrees1.append(degree_name1)
except:
  degree_name1="Unknown"
  degrees1.append(degree_name1)

try:
  industries.append(res['industry'])
except:
  industries.append("Unknown")

try:
  if res['summary']:
    summaries.append(res['summary'])
  else:
    summaries.append("Unknown")
except:
  summaries.append("Unknown")

# Load the saved model
model = load_model('model.h5')
a= [
    positions0[0],
    positions1[0],
    str(experinces0[0]),
    str(experinces1[0]),
    field_of_studies0[0],
    field_of_studies1[0],
    degrees0[0],
    degrees1[0],
    industries[0],
    skills[0],
    summaries[0]
]
new_text_data=[]
for i in a:
    if i != "Unknown":
        new_text_data.append(i)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(new_text_data)

# Pad sequences to have consistent length
max_sequence_length = 519
print(f"max_sequence------>>>>{max_sequence_length}")
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
# Preprocess the new text data
new_sequences = tokenizer.texts_to_sequences(new_text_data)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(new_padded_sequences)

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Map the predicted labels back to original labels
label_mapping={'Analyst (C)': 0, 'Questioner (CD)': 1, 'Captain (D)': 2, 'Encourager (Is)': 3, 'Stabilizer (SC)': 4, 
  'Supporter (S)': 5, 'Counselor (Si)': 6, 'Editor (Cs)': 7, 'Motivator (I)': 8, 'Driver (Di)': 9,
  'Planner (Sc)': 10, 'Skeptic (Cd)': 11, 'Architect (Dc)': 12, 'Driver (Di);': 13, 
  'Influencer (Id)': 14, 'Harmonizer (IS)': 15, 'Initiator (DI)': 16}
reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}
predicted_labels = [reverse_label_mapping[label] for label in predicted_labels]

# Print the predictions
for text, label in zip(new_text_data, predicted_labels):
    print(f"Text: {text}\tPredicted Label: {label}")
print(max(set(predicted_labels), key = predicted_labels.count))

max_sequence------>>>>519
1/1 [==============================] - 1s 1s/step
Text: Frontend mentor 	Predicted Label: Supporter (S)
Text: Fullstack | javaScript developer	Predicted Label: Analyst (C)
Text: 0	Predicted Label: Supporter (S)
Text: Petroleum Engineering	Predicted Label: Analyst (C)
Text: Oil and Gas	Predicted Label: Captain (D)
Text: Master of Science - MS	Predicted Label: Analyst (C)
Text: Bachelor's degree	Predicted Label: Editor (Cs)
Text: Computer Software	Predicted Label: Supporter (S)
Text: ['Python (Programming Language)', 'Django', 'REST APIs', 'Problem Solving', 'Responsive Web Design', 'Front-End Development', 'Firebase', 'Web Development', 'Communication', 'Contract Negotiation', 'Construction', 'Management', 'Analytical Skills', 'Software as a Service (SaaS)', 'Translation', 'Technical Translation', 'English Translation', 'Russian Translation', 'Redux.js', 'Git']	Predicted Label: Analyst (C)
Text: I am a skilled Frontend Developer with over 1.5 years of experienc

In [15]:
predicted_labels

['Supporter (S)',
 'Motivator (I)',
 'Harmonizer (IS)',
 'Questioner (CD)',
 'Counselor (Si)',
 'Supporter (S)',
 'Harmonizer (IS)',
 'Supporter (S)',
 'Harmonizer (IS)',
 'Influencer (Id)']

In [ ]:
# new_text_data = [
#     positions0[0],
#     positions1[0],
#     str(experinces0[0]),
#     str(experinces1[0]),
#     field_of_studies0[0],
#     field_of_studies1[0],
#     degrees0[0],
#     degrees1[0],
#     industries[0],
#     skills[0],
#     # str(influencer[0]),
#     # country[0],
#     summaries[0]

# ]


In [59]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model.h5')
a= [
    positions0[0],
    positions1[0],
    str(experinces0[0]),
    str(experinces1[0]),
    field_of_studies0[0],
    field_of_studies1[0],
    degrees0[0],
    degrees1[0],
    industries[0],
    skills[0],
    summaries[0]
]
new_text_data=[]
for i in a:
    if i != "Unknown" and i != None:
        new_text_data.append(i)

# Preprocess the new text data
new_sequences = tokenizer.texts_to_sequences(new_text_data)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(new_padded_sequences)

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Map the predicted labels back to original labels
reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}
predicted_labels = [reverse_label_mapping[label] for label in predicted_labels]

# Print the predictions
for text, label in zip(new_text_data, predicted_labels):
    print(f"Text: {text}\tPredicted Label: {label}")
print(max(set(predicted_labels), key = predicted_labels.count))


1/1 [==============================] - 1s 1s/step
Text: Sustainability Partner to Boards - Director Marketing & Sales	Predicted Label: Encourager (Is)
Text: Founder	Predicted Label: Captain (D)
Text: 14	Predicted Label: Driver (Di);
Text: Marketing	Predicted Label: Motivator (I)
Text: Post-doctorate degree in Brand Management	Predicted Label: Architect (Dc)
Text: Environmental Services	Predicted Label: Captain (D)
Text: ['Leadership Development', 'Investments', 'Strategy', 'Leadership', 'Sustainability Consulting', 'Marketing Strategy', 'Business Strategy', 'Marketing', 'B2B', 'New Business Development', 'Business Development', 'Marketing Management', 'Entrepreneurship', 'Start-ups', 'Project Planning', 'Supply Chain', 'Management Consulting', 'Change Management', 'FMCG', 'Strategic Planning']	Predicted Label: Motivator (I)
Text: Committed to help leaders at all levels transition their organisations towards businesses that are a force for good... leaving no footprint.

Main fields of e

In [46]:
max(set(predicted_labels), key = predicted_labels.count)

'Editor (Cs)'

In [45]:
predicted_labels

['Editor (Cs)',
 'Supporter (S)',
 'Supporter (S)',
 'Editor (Cs)',
 'Editor (Cs)',
 'Captain (D)',
 'Editor (Cs)']

In [26]:
# positions0[0]
# positions1[0]
# str(experinces0[0])
# str(experinces1[0])
# field_of_studies0[0]
# field_of_studies1[0]
# degrees0[0]
# degrees1[0]
# industries[0]
# skills[0]
# str(influencer[0])
# country[0]
summaries[0]

In [27]:
summaries

[None]

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')
cols = ['position 1', 'position 2', 'field of studies 1', 'experince 1', 'experince 2', 'field of studies 2',
        'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']

for col in cols:
    data[col] = data[col].fillna("Unknown")

# Convert non-string values to strings in the feature columns
text_features = data[
    ['position 1', 'position 2', "experince 1", "experince 2", 'field of studies 1', 'field of studies 2',
     'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country', 'summary']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.1, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100, kernel_regularizer=l2(0.01))(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)  # Add a dropout layer for regularization
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
learning_rate = 0.001  # Decrease the learning
# Compile the model
learning_rate = 0.001  # Decrease the learning rate

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Apply early stopping to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_text, y_train,
                    validation_data=(X_test_text, y_test),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping])

# Save the model
model.save('model3.h5')


523


NameError: name 'tf' is not defined

In [ ]:
# 1MAbPt8f4U14J6XsT5WHxrTdgo1VpPAF

In [ ]:
r5jcbZqwAKUU6MFDxRnMAfFUf6p18KT9

In [3]:
import datetime
import requests
import numpy as np
import json
import pandas as pd
# from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
data=pd.read_csv("./partly_45878_reach_50.csv")
for i in range(5000, 5010):
  link=data['links'][i]
  id = link.split('/')[4]
  url = "https://api.iscraper.io/v2/profile-details"

  payload = {
    'profile_id': id,
  }

  headers = {
    'X-API-KEY': '1MAbPt8f4U14J6XsT5WHxrTdgo1VpPAF',
  }

  response = requests.post(url, json=payload, headers=headers)
  res=response.json()

  with open(f"./datas/json_responses_50k/{i}.json", "w") as json_file:
      json.dump(res, json_file, indent=4)
  print(i)

5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
